In [3]:
!pip install py-algorand-sdk
!pip install python-dateutil
!pip install pandas
!pip install typing-extensions
!pip install requests
!pip install aiohttp
!pip install msgpack


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
from dataclasses import dataclass
from typing import Optional, List, Dict
from datetime import datetime, timedelta
from algosdk.v2client import indexer
import logging
import csv
import os
import json
from pathlib import Path

In [6]:
# Configuración del logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Clase TransactionConfig

Clase que encapsula los parámetros de configuración para análisis de transacciones en Algorand usando dataclasses.

Parámetros por defecto:
- Indexer API: mainnet-idx.algonode.cloud
- Directorio de reportes: "reports"
- Precisión decimal: 6 dígitos

Principales funciones:
- `validate_dates()`: Convierte fechas YYYY-MM-DD a formato RFC3339 para el indexer.
- Manejo de errores para formatos de fecha incorrectos.
- Soporte para rangos de fecha opcionales.
- Ajuste automático del rango temporal (end_date hasta 23:59:59).

Uso principalmente como contenedor de configuración para asegurar consistencia en los parámetros de análisis de transacciones.

In [7]:
@dataclass
class TransactionConfig:
    """
    Clase de configuración para el análisis de transacciones.
    
    Attributes:
        indexer_address (str): Dirección del indexer de Algorand
        indexer_token (str): Token de autenticación para el indexer
        start_date (str): Fecha inicial para el filtro (formato: 'YYYY-MM-DD')
        end_date (str): Fecha final para el filtro (formato: 'YYYY-MM-DD')
        output_dir (str): Directorio para guardar los reportes
        decimal_precision (int): Número de decimales para los montos
    """
    indexer_address: str = "https://mainnet-idx.algonode.cloud"
    indexer_token: str = ""
    start_date: Optional[str] = None
    end_date: Optional[str] = None
    output_dir: str = "reports"
    decimal_precision: int = 6

    def validate_dates(self) -> tuple[Optional[str], Optional[str]]:
        """
        Valida y convierte las fechas al formato requerido por el indexer.
        
        Returns:
            tuple: (fecha_inicio, fecha_fin) en formato RFC3339
        """
        try:
            if self.start_date:
                start = datetime.strptime(self.start_date, '%Y-%m-%d')
                start_rfc = start.isoformat() + 'Z'
            else:
                start_rfc = None

            if self.end_date:
                end = datetime.strptime(self.end_date, '%Y-%m-%d')
                end = end + timedelta(days=1) - timedelta(seconds=1)
                end_rfc = end.isoformat() + 'Z'
            else:
                end_rfc = None

            return start_rfc, end_rfc
        except ValueError as e:
            logger.error(f"Error en el formato de las fechas: {e}")
            raise ValueError("Las fechas deben estar en formato 'YYYY-MM-DD'")

# WalletAnalyzer: Analizador Detallado de Transacciones

Clase especializada para análisis profundo de una wallet en Algorand:

## Funcionalidades Principales:

1. Validación y Configuración:
   - Valida formato de dirección (58 caracteres)
   - Configura cliente indexer
   - Gestiona directorio de salida

2. Obtención de Transacciones:
   - Fetch paginado de transacciones
   - Filtros por rango de fechas
   - Manejo de errores de conexión

3. Procesamiento de Transacciones:
   - Soporte para ALGO y ASAs
   - Clasificación: enviadas/recibidas
   - Decodificación de notas
   - Normalización de montos (6 decimales)

4. Generación de Reportes:
   - CSV con timestamp
   - Campos detallados por transacción
   - Información completa de movimientos

El sistema está diseñado para análisis detallado y trazabilidad de movimientos en carteras específicas de Algorand.

In [8]:
class WalletAnalyzer:
    """
    Clase principal para analizar transacciones de una wallet en Algorand.
    
    Attributes:
        wallet_address (str): Dirección de la wallet a analizar
        config (TransactionConfig): Configuración del análisis
        client (IndexerClient): Cliente del indexer de Algorand
    """
    
    def __init__(self, wallet_address: str, config: TransactionConfig):
        """
        Inicializa el analizador de wallet.
        
        Args:
            wallet_address (str): Dirección de la wallet
            config (TransactionConfig): Configuración del análisis
        """
        self.validate_address(wallet_address)
        self.wallet_address = wallet_address
        self.config = config
        self.client = indexer.IndexerClient(
            indexer_token=config.indexer_token,
            indexer_address=config.indexer_address
        )
        self._setup_output_directory()

    @staticmethod
    def validate_address(address: str) -> None:
        """
        Valida el formato de la dirección de la wallet.
        
        Args:
            address (str): Dirección a validar
            
        Raises:
            ValueError: Si la dirección no cumple con el formato esperado
        """
        if not address or len(address) != 58:
            raise ValueError("La dirección de la wallet no es válida")
        
    def _setup_output_directory(self) -> None:
        """Crea el directorio de salida si no existe."""
        Path(self.config.output_dir).mkdir(parents=True, exist_ok=True)
        
    def fetch_transactions(self) -> List[Dict]:
            """
            Obtiene todas las transacciones de la wallet aplicando los filtros de fecha.
            
            Returns:
                List[Dict]: Lista de transacciones
            """
            transactions = []
            next_token = None
            start_time, end_time = self.config.validate_dates()

            try:
                while True:
                    response = self.client.search_transactions_by_address(
                        address=self.wallet_address,
                        next_page=next_token,
                        start_time=start_time,
                        end_time=end_time
                    )
                    
                    current_transactions = response.get('transactions', [])
                    if not current_transactions:
                        break
                    
                    transactions.extend(current_transactions)
                    next_token = response.get('next-token')
                    
                    logger.info(f"Obtenidas {len(current_transactions)} transacciones adicionales")
                    
                    if not next_token:
                        break
                        
            except Exception as e:
                logger.error(f"Error al obtener transacciones: {e}")
                raise RuntimeError(f"Error en la conexión con el indexer: {e}")
                
            return transactions

    def process_transactions(self, transactions: List[Dict]) -> List[Dict]:
        """
        Procesa y formatea las transacciones para el reporte.
        
        Args:
            transactions (List[Dict]): Lista de transacciones crudas
            
        Returns:
            List[Dict]: Lista de transacciones procesadas
        """
        processed_transactions = []
        
        for txn in transactions:
            try:
                processed_txn = self._process_single_transaction(txn)
                if processed_txn:
                    processed_transactions.append(processed_txn)
            except Exception as e:
                logger.warning(f"Error procesando transacción {txn.get('id', 'unknown')}: {e}")
                continue
                
        return processed_transactions

    def _process_single_transaction(self, txn: Dict) -> Optional[Dict]:
        """
        Procesa una única transacción.
        
        Args:
            txn (Dict): Transacción individual
            
        Returns:
            Optional[Dict]: Transacción procesada o None si debe ser ignorada
        """
        tx_type = txn.get('tx-type')
        
        # Procesamos diferentes tipos de transacciones
        if tx_type == 'axfer':  # Transferencia de activos
            return self._process_asset_transfer(txn)
        elif tx_type == 'pay':  # Transferencia de Algos
            return self._process_payment(txn)
        else:
            logger.debug(f"Tipo de transacción no procesada: {tx_type}")
            return None

    def _process_asset_transfer(self, txn: Dict) -> Dict:
        """
        Procesa una transferencia de activos (ASA).
        
        Args:
            txn (Dict): Transacción de transferencia de activos
            
        Returns:
            Dict: Información procesada de la transacción
        """
        asset_transfer = txn.get('asset-transfer-transaction', {})
        receiver = asset_transfer.get('receiver')
        
        return {
            'fecha': datetime.fromtimestamp(txn.get('round-time', 0)).strftime('%Y-%m-%d %H:%M:%S'),
            'tipo': "Recibido" if receiver == self.wallet_address else "Enviado",
            'asset_id': asset_transfer.get('asset-id'),
            'monto': asset_transfer.get('amount', 0) / (10 ** self.config.decimal_precision),
            'remitente': txn.get('sender'),
            'receptor': receiver,
            'tx_id': txn.get('id'),
            'ronda': txn.get('confirmed-round'),
            'nota': self._decode_note(txn.get('note'))
        }

    def _process_payment(self, txn: Dict) -> Dict:
        """
        Procesa una transferencia de Algos.
        
        Args:
            txn (Dict): Transacción de pago
            
        Returns:
            Dict: Información procesada de la transacción
        """
        payment = txn.get('payment-transaction', {})
        receiver = payment.get('receiver')
        
        return {
            'fecha': datetime.fromtimestamp(txn.get('round-time', 0)).strftime('%Y-%m-%d %H:%M:%S'),
            'tipo': "Recibido" if receiver == self.wallet_address else "Enviado",
            'asset_id': 'ALGO',
            'monto': payment.get('amount', 0) / 1_000_000,  # Algos siempre tienen 6 decimales
            'remitente': txn.get('sender'),
            'receptor': receiver,
            'tx_id': txn.get('id'),
            'ronda': txn.get('confirmed-round'),
            'nota': self._decode_note(txn.get('note'))
        }

    @staticmethod
    def _decode_note(note: Optional[str]) -> str:
        """
        Decodifica la nota de la transacción si existe.
        
        Args:
            note (Optional[str]): Nota codificada en base64
            
        Returns:
            str: Nota decodificada o cadena vacía
        """
        if not note:
            return ""
        try:
            import base64
            decoded = base64.b64decode(note).decode('utf-8')
            return decoded
        except Exception:
            return ""
    
    def generate_report(self, transactions: List[Dict], wallet_address: str) -> str:
        """
        Genera el reporte CSV con las transacciones.
        
        Args:
            transactions (List[Dict]): Lista de transacciones procesadas
            
        Returns:
            str: Ruta del archivo generado
        """
        if not transactions:
            logger.warning("No hay transacciones para generar el reporte")
            return ""

        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"{self.config.output_dir}/wallet_transactions_{wallet_address}_{timestamp}.csv"
        
        try:
            with open(filename, mode='w', newline='', encoding='utf-8') as file:
                writer = csv.DictWriter(file, fieldnames=transactions[0].keys())
                writer.writeheader()
                writer.writerows(transactions)
                
            logger.info(f"Reporte generado exitosamente: {filename}")
            return filename
        except Exception as e:
            logger.error(f"Error generando el reporte: {e}")
            raise

# Función Principal del Analizador

Este código implementa el flujo completo de análisis:

Configuración:
- Rango temporal: 2023-01-01 a 2024-12-27
- Directorio: algorand_reports
- Wallet específica a analizar

Proceso:
1. Inicializa analizador
2. Obtiene transacciones
3. Procesa datos
4. Genera reporte CSV

Incluye manejo de errores y logging para monitoreo de ejecución.

El ejemplo de uso demuestra la API completa, desde configuración hasta generación de reporte personalizado.

In [9]:
def main():
    """
    Función principal que ejecuta el análisis de transacciones.
    """
    # Configuración
    config = TransactionConfig(
        start_date="2023-01-01",  # Ajusta estas fechas según necesites
        end_date="2024-12-27",
        output_dir="algorand_reports"
    )
    
    # Lista de direcciones de wallet a analizar
    wallet_addresses = [
        "N2C374IRX7HEX2YEQWJBTRSVRHRUV4ZSF76S54WV4COTHRUNYRCI47R3WU",
        "MTCEM5YJJSYGW2RCXYXGE4SXLSPUUEJKQAWG2GUX6CNN72KQ3XPJCM6NOI",
        "NE5EGI4JA5EFJV3AVD4OPYNBJYN66ZKOWKOXADZKZMZ3DFGQGY3VEKHMFU",
        "FB3ILSDVDSECHNC25SZCOCQVKVQC27MF54S42KQ2H7THP7MU2WEBNDJYJY",
        "HEOQ3S6V47RFLU2RZ5GTQYJBEFRL54UWZ77PNUBNTDVSXIPYOPE2XZJSLE",
        "J63NZUR3S4LJC2RPBSOXGQ75DUVQEBTIFUWL2R7FCQES24HFLQAFB6WDAY",
        "5P4IEQSZBRAQHE4WQIIMYQGSVD4N6ZWM7TEJ3QXXMZIL4BR3MV6BFQYTUQ",
        "ZDEFFZZ4NAWFP525TOGNWICWOXKOF74DFMYCE6BMMZIZVX5NNPYKN4SRRA",
        "2R5FTTVDIAQ55I5SPW5BE6R2SVYY45O5W64XGIVBLHQYWMZARRXTO4VIHQ",
        "LYKPESAJWNRAV7XADUHBKLADBK47HWKIM4W3K7ICKXXEURMP2AJLYLFDF4",
        "JJT4QS7NTMCKVSLJUZ6BQLV44MLQ4F5OWFJP26CN3BYTPORMMRO5AB6QCA",
        "53B5Q2CMPW2HZDZYKXECOMQGZDWNIT4W75B3D5XAT3ABPDCMCQT6RVHJGI",
        "WZY5ZGJ57YOVJ2TCMFXIH6TNLDC2FI2UJSWGG4XN5RK4XVQX4MMOIZD2AU",
        "BX7UST4VVWQQPPAVPMPFW76QRKXRLQ3UVYABWBTHOAX2AN5Q5OGW2X55AQ",
        "KQ6FVR6PTHMV7QV47LBSR3LVM3EWSNHJPTWQ7GUEFLI2NGKC3RAULQ27K4",
        "DLRY7TLD3KXB4G6IAIW5KGUZEVIDBS2OZ3LR7RFSJD2KEUDKWFXEBPK3HU",
        "WD5EDLJCLRWFIBEUCT43J6QM3LGVNSYXUSMWGUKKUMJBZ65IKQWIAV2LIY",
        "UI6AGNWYGQD6HNQVEZ5ZTAQR27FHZ45VMLTS7LTSF2IZGAJMOIF4V4EEVU",
        "IVBHJFHZWPXRX2EA7AH7Y4UTTBO2AK73XI65OIXDEAEN7VO2IHWXKOKOVM",
        "T6MXHXMGXURKJ7AD5NCB4IYPLWXYXKKFLLO5KAPJ56GKS7BFXHNHOODZCM",
        "FFLRE4VZK35KAU65VLBRMR4KZVHE4S2TEXJJ6K4POZJZXUND3V2V6DVJNU",
        "5GW5VO4JNTHXVH2DYV7HAKBKGQFSCAI4MBUD5EN3FLBGLF4KKRXJE24ASI",
        "3SORALOCWWTHAAGEZIOB5MIV2G3KAWUN2LQI4ZBWPLEDZANPIVTHFHDGZ4",
        "ZVMOZVZJK64NEYDPUDGGC52NI6HOX2LUQVIWYCQTJ2DFXRGPL72C2BQYNM",
        "2ZPNLKXWCOUJ2ONYWZEIWOUYRXL36VCIBGJ4ZJ2AAGET5SIRTHKSNFDJJ4",
        "GH27QMBSQZF3O6J7SGXQRQMUQD264HNXZ7R6OXDPIOR5XOOE22UBWUXZI4",
        "LRDSA35VTQL3ORSNBCM7TPGMYZKWK3OTRWSKD6NSXOKAOIPHQXJFEKNS2Q",
        "EU6CHYSH7ZXLJQAPPIN6W3KS7VAURYZCB5P3ZCXMYCWNJF6V5RTVL2UPHU",
        "4H5UNRBJ2Q6JENAXQ6HNTGKLKINP4J4VTQBEPK5F3I6RDICMZBPGNH6KD4",
        "TBN2J7U3J5D4I7R2EK7XIBFNTEGVLHNORAXQ6YBJY5IVNY5IIKOXSJRYCE",
        "L5BLJ4FNK6FNM7V5NUVT5QI6NQAERLLHYT24XH6RS2DUC4WDHPM5LOLGBY",
        "VEJGTLTKNT3VGLG2GVB2LMXC55WYW6J6WPZ76XTY2Y46TRJQOORWERYXYE",
        "XBYLS2E6YI6XXL5BWCAMOA4GTWHXWENZMX5UHXMRNWWUQ7BXCY5WC5TEPA",
        "2JGGWKOIKYZB4HLG2X5DWHD5EWCUOQR7DC6VOEMWELIVNVVAF3BEUWJR7Y",
        "LHVWNRKGGOTSSDYK4P4WKXTHZI5SAFKUO5ALAW7NJ6G76RG4UXLBCWN5LQ",
        "NRDDQ7MFRTUTMDAP4CBXDQ2IVP5VSLKDASADLLANYLFIKR7NQOGOUINYM4",
        "62UUOSMOMD6XOSRROCIIMVVF2VX6N4CMVLCUFUVWV4Q4T4BHD7ETFNWMOI",
        "O4N25TS4Z5SC34VZ6R6RU74PCEIUTJFSDSKETNDYU4CXI3C2BFYXCYEKAU",
        "GJGK42UVZK4IDKN5MGP53A6FJEHRI52PI4E3BBJZRZCQZ666BKYILYXI2E",
        "R3T4HFV3QXOHCZHRSQDJATCI2E7KHFA4BAWG5C6CGS3MGFNUFSI5FAYEN4",
        "VZBMOTCMEHRITULBNLCWHA5U62UUXLQZMLVM6HPBPCMIM2YOMHDQRTK64E",
        "OM2NLTOCWVDGX5XI6ETIQPW2CEILSEJFOC4NX5TJCTU6WMNC2KT2OUCT4M",
        "VAOTJJLJP54QIKGCCFNJZVNHXXFZUZ3AAXCVGX5LRDQXQOUZRWBFBASUDQ",
        "A3MYPLCU5ZVXDJEAXAVV2SAUWJ4INWGPYOKM3FWOGUASOJ7V2L6DTC5YN4",
        "5EYP2467BNWUIE773T57P3VZ5RRMWUFCPSYOOUHVFP4AY5HSZ7MKECKTKI",
        "5WCIZNGQQT747WX3RTQIBJHOMJTQRUQBBH3PMK4YLP2X33AICJEUTL6F2E",
        "R7ALVPEQRGECJK33LANXMPENWYALTAZQTSGNCTEQPKBSBDI5KO252SAK64",
        "P73D4BTS5B55FBV5Q4K2JZNTJOOMACZY4TFYKLCHPCOGLUPGDBXDBVSS7E",
        "YX5KZSZT27L7WZAW7TNONVDZHQQAURJKT4BPRS364KTH2DGMEKLLFOPK3U",
        "IHYR5OZGAIRSCDCNQJVFOPAOJT2SPG3YXAE3GGJPZRI6JV2GQSJAYG5NUY",
        "OFB2SMWDDB3V4RYRSC6EXJYB2RYFVPWGYYEGAOLK52TN72G2DELKVENV44",
        "A66JRYUOU523Z4MU53AJL3YAEHESH3KMVV7OJI4SMFRVIDNNVDK2LHSL4Y",
        "7LXGH4WPEXJ3HOPON5BDHJSJ7NWPBAELDOWLINCMMW2GOQWOSSNP76D4ZA",
        "MKZIWVBDBZV7UK6XQY3DFLYSBLSJWCDHDJWK3JAHWCFMNJOH4ZXQSMOUCE",
        "2JBQSMWAZANNM5U2P2DPCBVJ7VYLRT7LA2AOP7JGUF4HIBV6XBA3TBGXGA",
        "E4VMOYKWJCTKYVY447ALPOZMORITQYQVZVSYJ4ABLCCAAX4ZE6RNFDXZPI",
        "HRLD25IMT2Q4UPYOEUZIWHDI3ELCUIC5NLNC75O2NE7OLDJK7GZXDIK5QQ",
        "EG6JXQ3TQBWRSTR3OEDUS5RTPLMA4KTMJIV3N6DO7XN2XRKIFEN64DY3BU",
        "WDWBXGJIXO3N6A7AZ25XU4UX5Q3FJJ5CCKFCUEUWE75ZF5I6H47X37EY6M",
        "ASHPGJPHCZMSVBBUVFOVTSIYFKCR26STPOVPU4GWACKTZT35ZQYBSE775U",
        "W425PRGCPQFAKWSN7D2QKGKBHUWEVV4Z5PWLKUTIGYULGEU5ZDXOAKETC4",
        "5I7NE6QCQYZUUKKMZ2HLB2AWO7NDUXNKVFACAF46NEP3TA6ZHAYOHIEVMU",
        "7GPQ36Y43EF77SDEMPMGCFIWBT3CRP3U6VMT7SK42X42ZUOXVS7TD3KXAA",
        "WW5MEAVIVE3X4N6ZED4CEXUKAAEBACMS2HSGG7GIOXKHWCWKCAGAWDSGL4",
        "KHGLFYRM52BT6R5CSREB7BNUOSR3FUEHCWLIXKWM44SD2RZMSXDPZS77U4",
        "HMKN4BCGCDUFWIPOIKDBUTC6RXFKOKOB6TWO25ETRHTKJFTJZPD6Y3BOLQ",
        "OTW6HXVSWXDRBV2F7RK5YHYBERVYUYQ4KSVRW5K4ECBONAJBSFHHDSP764",
        "SXNDBVZXTLF6JJVER3VZV5XMGV6SGDNANHATZZIKHWVF5RV2JVPQ7NB56E",
        "7ZUHI3E5Z7V5XJTQISK76KXG4O7KJY66VD7LFIX2EZ5Z5VOVZBONZDAVWM",
        "QYXDGS2XJJT7QNR6EJ2YHNZFONU6ROFM6BKTBNVT63ZXQ5OC6IYSPNDJ4U",
        "XMVGCC27ZCGWKOQYZDE5WESGYS4V7CRTQVIAZETJNNV7U3QTHJ6CUZ62PQ",
        "C4PZ7RSTYDAAEJE3PIMJW3BRYKNY6OHYZVZO6OLXPZKBMXVWDB4PBECTZI",
        "OMHYS6DGAS2GQIMALPYPSMNDQ2735J7Z76RQFH7KP2MIPDHXOZYVV3TVO4",
        "V6F7XZUE6G4ODLE47CY32PLDLEC6LELBE7JWQX6RMBJBPFMZ5UOM3WXMWM",
        "EIATQQ3TAKBR67FOEMPPSOA2ZYXO5FNXQQDUPRFFISS2Y6AO5HMJBFNRXE",
        "JHCVUBWAJHEMNPBZDNYVXK6DNRFUJWNLWXDDLLBOLIL3KFCQAOI3XVMWOA",
        "PZ7TILKNWVGJTAWF7YSBYDHT44SN6NQJXHI55ODGJXGXHGB64BH6MBDGFY",
        "UTFJWM5YP6TSR66YHRYBNDZLCYTTSMLDBKKFTEE723HT7GJBGNCWD6UDC4",
        "SJYNEQ36GVJBQ3YRZTVQGMFQDHRZKK3SGXF224WKNP7S3NAU563RNPUMWA",
        "UPBR75UVEX4UVBKLLXDEVF5OI6SN426G7IEEZFM2QYMXQHPHTX5FZUT2MA",
        "DEK6IBQKSX5JPKUSIDCLRUEXTAJLSEM5EFZWXDDYYVPKBH3JOS4NPT477U",
        "FRWQZO4A6NZKANEYYWAHZIBHJ46T2LVRFACCLHUY3JQAJLUIWNQQLOQ26A",
        "ASDNGCOKPWDF3IFVNGVYTRC4EBU4RGFBIX4PVFMBVUM3T4WIKID5GABY3Y",
        "UFEO4JIEEJD73NJM6FPRD57USABORDHLDBJPBNIZBHB3TJTA6CBNSAPI4E",
        "A6B5AKODYAMHTQ7B4PJAMOLDLZFMAB4QOXJJP6I3FC7YIPOPD6PL7ZYRCA",
        "O4WKNVA25QYYMN3NW534CWE65PPIMU56TNB5ZJIKGOQHBXEFU6QACUJWOQ",
        "P2W7L5OERZ53KV4LJOCV3HR75ZPA6BDLHCAL27IQSIE2WDFGRFEGL524VA",
        "PRNU5BPKXY5G322XPP7D7R7NH7MFFUUO44522T3P5OT64DB56JJ5ZQJ4LA",
        "I63NBAY2FYICKXSF7XFJRUV3CQDB756EDQ245D47Z34LTEFP7JYXS56QTM",
        "3XQ3RJXTG2VDOV45ZKYIGEVXP2VF2HDZJX6I3NUCIJHXGHYEDA74LRSQ5U",
        "LSAH7NBFXSUP2RSNS4UDA6KA3THYIS5CAWVE7IG4N45FAEGQOXYCXSL4TI",
        "C2QRHHR3K7KSLNQV4BJCD6UVM5OJSSXTV2YQR6DOYVL3X7ZGC2BBZRFKEE",
        "ALMEZURUOG7R2QKPZIPNQAFANRQEZU4J22ZJYW6HARU4IZCZGEHNTZMO6U",
        "B2465RWQTTRXELPWRZENQIQLBPDTQQZJGEQXLWQHQTBHUMSD2RKA7CAWEM",
        "B23XLXSA3MBU3SLP5EF2HE7JRR2UMFNKZBIDRWQPGLGCNDNCABWHMDXPMQ",
        "K4F7HD3ZEQZ7SG5XDSXJSC6HVXXCQQQK6E47MRA5ZUJ2I7Y54G4LUZKMUY",
        "ILZP3VLXPZOMULC5XMR5UUL5LMBGXX6TT6IURYFZ7Q6KX54INCUISZH2JQ",
        "22LRYAZSTGCB7DXQUKOWA3JG4ZP2DZJ3H6R5ZCVE5XBIXLK2VPST5CY2J4",
        "3NGBML54PC7AJATJGW5BXMF6ZOFO2V4VTKKRTS3EAZXTVVHSPXTNV2GKEA",
        "XUPBGF6OXIRVIGU2VHHYJFI4JEHLLIPNLMWNCSUZ7F44KYFRPV52ULIYNI"
    ]
    
    try:
        for wallet_address in wallet_addresses:
            # Inicializar el analizador
            analyzer = WalletAnalyzer(wallet_address, config)
            
            # Obtener transacciones
            logger.info("Obteniendo transacciones...")
            transactions = analyzer.fetch_transactions()
            
            # Procesar transacciones
            logger.info("Procesando transacciones...")
            processed_transactions = analyzer.process_transactions(transactions)
            
            # Generar reporte
            logger.info("Generando reporte...")
            report_path = analyzer.generate_report(processed_transactions, wallet_address)
            
            if report_path:
                print(f"\nReporte generado exitosamente: {report_path}")
                print(f"Total de transacciones procesadas: {len(processed_transactions)}")
            else:
                print("\nNo se encontraron transacciones para el período especificado.")
                
    except Exception as e:
        logger.error(f"Error en la ejecución: {e}")
        print(f"\nError: {e}")

if __name__ == "__main__":
    main()

2024-12-30 15:21:59,889 - __main__ - INFO - Obteniendo transacciones...
2024-12-30 15:22:01,563 - __main__ - INFO - Obtenidas 1000 transacciones adicionales
2024-12-30 15:22:02,403 - __main__ - INFO - Obtenidas 1000 transacciones adicionales
2024-12-30 15:22:03,488 - __main__ - INFO - Obtenidas 1000 transacciones adicionales
2024-12-30 15:22:04,554 - __main__ - INFO - Obtenidas 1000 transacciones adicionales
2024-12-30 15:22:05,390 - __main__ - INFO - Obtenidas 1000 transacciones adicionales
2024-12-30 15:22:06,164 - __main__ - INFO - Obtenidas 1000 transacciones adicionales
2024-12-30 15:22:07,006 - __main__ - INFO - Obtenidas 1000 transacciones adicionales
2024-12-30 15:22:07,920 - __main__ - INFO - Obtenidas 1000 transacciones adicionales
2024-12-30 15:22:08,651 - __main__ - INFO - Obtenidas 1000 transacciones adicionales
2024-12-30 15:22:09,481 - __main__ - INFO - Obtenidas 1000 transacciones adicionales
2024-12-30 15:22:09,927 - __main__ - INFO - Obtenidas 179 transacciones adicio


Reporte generado exitosamente: algorand_reports/wallet_transactions_N2C374IRX7HEX2YEQWJBTRSVRHRUV4ZSF76S54WV4COTHRUNYRCI47R3WU_20241230_152210.csv
Total de transacciones procesadas: 10177


2024-12-30 15:22:11,161 - __main__ - INFO - Obtenidas 266 transacciones adicionales
2024-12-30 15:22:11,621 - __main__ - INFO - Procesando transacciones...
2024-12-30 15:22:11,633 - __main__ - INFO - Generando reporte...
2024-12-30 15:22:11,642 - __main__ - INFO - Reporte generado exitosamente: algorand_reports/wallet_transactions_MTCEM5YJJSYGW2RCXYXGE4SXLSPUUEJKQAWG2GUX6CNN72KQ3XPJCM6NOI_20241230_152211.csv
2024-12-30 15:22:11,645 - __main__ - INFO - Obteniendo transacciones...



Reporte generado exitosamente: algorand_reports/wallet_transactions_MTCEM5YJJSYGW2RCXYXGE4SXLSPUUEJKQAWG2GUX6CNN72KQ3XPJCM6NOI_20241230_152211.csv
Total de transacciones procesadas: 264


2024-12-30 15:22:12,118 - __main__ - INFO - Obtenidas 13 transacciones adicionales
2024-12-30 15:22:12,540 - __main__ - INFO - Procesando transacciones...
2024-12-30 15:22:12,543 - __main__ - INFO - Generando reporte...
2024-12-30 15:22:12,547 - __main__ - INFO - Reporte generado exitosamente: algorand_reports/wallet_transactions_NE5EGI4JA5EFJV3AVD4OPYNBJYN66ZKOWKOXADZKZMZ3DFGQGY3VEKHMFU_20241230_152212.csv
2024-12-30 15:22:12,550 - __main__ - INFO - Obteniendo transacciones...


...


Reporte generado exitosamente: algorand_reports/wallet_transactions_WD5EDLJCLRWFIBEUCT43J6QM3LGVNSYXUSMWGUKKUMJBZ65IKQWIAV2LIY_20241230_152224.csv
Total de transacciones procesadas: 67


2024-12-30 15:22:24,829 - __main__ - INFO - Obtenidas 1000 transacciones adicionales
2024-12-30 15:22:25,696 - __main__ - INFO - Obtenidas 1000 transacciones adicionales
2024-12-30 15:22:26,750 - __main__ - INFO - Obtenidas 1000 transacciones adicionales
2024-12-30 15:22:27,553 - __main__ - INFO - Obtenidas 1000 transacciones adicionales
2024-12-30 15:22:28,437 - __main__ - INFO - Obtenidas 1000 transacciones adicionales
2024-12-30 15:22:29,399 - __main__ - INFO - Obtenidas 1000 transacciones adicionales
2024-12-30 15:22:30,926 - __main__ - INFO - Obtenidas 1000 transacciones adicionales
2024-12-30 15:22:32,187 - __main__ - INFO - Obtenidas 1000 transacciones adicionales
2024-12-30 15:22:33,026 - __main__ - INFO - Obtenidas 1000 transacciones adicionales
2024-12-30 15:22:33,890 - __main__ - INFO - Obtenidas 1000 transacciones adicionales
2024-12-30 15:22:34,731 - __main__ - INFO - Obtenidas 1000 transacciones adicionales
2024-12-30 15:22:35,653 - __main__ - INFO - Obtenidas 1000 transa


Reporte generado exitosamente: algorand_reports/wallet_transactions_UI6AGNWYGQD6HNQVEZ5ZTAQR27FHZ45VMLTS7LTSF2IZGAJMOIF4V4EEVU_20241230_152246.csv
Total de transacciones procesadas: 22647


2024-12-30 15:22:48,156 - __main__ - INFO - Obtenidas 1000 transacciones adicionales
2024-12-30 15:22:48,911 - __main__ - INFO - Obtenidas 1000 transacciones adicionales
2024-12-30 15:22:49,776 - __main__ - INFO - Obtenidas 1000 transacciones adicionales
2024-12-30 15:22:50,594 - __main__ - INFO - Obtenidas 1000 transacciones adicionales
2024-12-30 15:22:51,389 - __main__ - INFO - Obtenidas 1000 transacciones adicionales
2024-12-30 15:22:52,154 - __main__ - INFO - Obtenidas 1000 transacciones adicionales
2024-12-30 15:22:52,906 - __main__ - INFO - Obtenidas 1000 transacciones adicionales
2024-12-30 15:22:53,699 - __main__ - INFO - Obtenidas 1000 transacciones adicionales
2024-12-30 15:22:54,567 - __main__ - INFO - Obtenidas 1000 transacciones adicionales
2024-12-30 15:22:55,360 - __main__ - INFO - Obtenidas 1000 transacciones adicionales
2024-12-30 15:22:56,154 - __main__ - INFO - Obtenidas 1000 transacciones adicionales
2024-12-30 15:22:56,946 - __main__ - INFO - Obtenidas 1000 transa


Error: Error en la conexión con el indexer: error while searching for transaction: ERROR: canceling statement due to statement timeout (SQLSTATE 57014)


# WhaleAnalyzer: Analizador de Comportamiento de Whales

Esta clase implementa análisis profundo de whales en Algorand:

## Funcionalidades Principales

1. `get_top_whales()`:
   - Identifica top 100 cuentas por balance
   - Balance mínimo configurable (default 100k ALGO)
   - Recopila datos base: balance, assets, métricas transaccionales

2. `analyze_whale_behavior()`:
   - Analiza comportamiento últimos 30 días
   - Métricas: transacciones, volúmenes, promedios
   - Diferencia entre envíos y recepciones

3. `generate_whale_report()`:
   - Genera reporte CSV con timestamp
   - Combina datos de balance y comportamiento
   - Campos: address, balance, assets, transacciones, volúmenes
   - Almacena en directorio configurado

## Estructura de Datos
- Balance en ALGO (dividido por 1_000_000)
- Volúmenes transaccionales
- Métricas de comportamiento
- Assets asociados

Los reportes permiten rastrear patrones de movimiento y concentración de fondos en la red Algorand.

In [24]:
class WhaleAnalyzer:
    def __init__(self, config: TransactionConfig):
        self.config = config
        self.client = indexer.IndexerClient(
            indexer_token=config.indexer_token,
            indexer_address=config.indexer_address
        )
        os.makedirs(self.config.output_dir, exist_ok=True)

    def get_top_whales(self, min_balance: int = 100_000) -> List[Dict]:
        try:
            accounts = []
            next_token = ""
            
            while True:
                response = self.client.accounts(
                    min_balance=min_balance * 1_000_000,
                    next_page=next_token if next_token else None,
                    limit=100
                )
                
                for account in response.get('accounts', []):
                    accounts.append({
                        'address': account['address'],
                        'balance': account['amount'] / 1_000_000,
                        'total_assets_value': 0,
                        'assets': account.get('assets', []),
                        'transaction_count': 0,
                        'total_volume': 0,
                        'avg_transaction_size': 0,
                        'sent_volume': 0,
                        'received_volume': 0
                    })
                
                next_token = response.get('next-token')
                if not next_token or len(accounts) >= 100:
                    break
            
            return sorted(accounts, key=lambda x: x['balance'], reverse=True)[:100]
            
        except Exception as e:
            logger.error(f"Error fetching accounts: {e}")
            return []

    def analyze_whale_behavior(self, whale_address: str, days: int = 30) -> Dict:
        end_time = datetime.now()
        start_time = end_time - timedelta(days=days)
        
        analyzer = WalletAnalyzer(whale_address, self.config)
        transactions = analyzer.fetch_transactions()
        processed_txns = analyzer.process_transactions(transactions)
        
        return {
            'address': whale_address,
            'transaction_count': len(processed_txns),
            'total_volume': sum(tx['monto'] for tx in processed_txns),
            'avg_transaction_size': sum(tx['monto'] for tx in processed_txns) / len(processed_txns) if processed_txns else 0,
            'sent_volume': sum(tx['monto'] for tx in processed_txns if tx['tipo'] == 'Enviado'),
            'received_volume': sum(tx['monto'] for tx in processed_txns if tx['tipo'] == 'Recibido'),
        }

    def generate_whale_report(self) -> str:
        whales = self.get_top_whales()
        if not whales:
            logger.error("No whales found")
            return ""
            
        # Procesar cada ballena
        processed_whales = []
        for whale in whales:
            behavior = self.analyze_whale_behavior(whale['address'])
            # Crear nuevo diccionario solo con los campos necesarios
            processed_whale = {
                'address': whale['address'],
                'balance': whale['balance'],
                'total_assets_value': whale['total_assets_value'],
                'transaction_count': behavior['transaction_count'],
                'total_volume': behavior['total_volume'],
                'avg_transaction_size': behavior['avg_transaction_size'],
                'sent_volume': behavior['sent_volume'],
                'received_volume': behavior['received_volume']
            }
            processed_whales.append(processed_whale)
        
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = os.path.join(self.config.output_dir, f"whale_analysis_{timestamp}.csv")
        
        with open(filename, 'w', newline='') as f:
            writer = csv.DictWriter(f, fieldnames=[
                'address', 'balance', 'total_assets_value', 
                'transaction_count', 'total_volume', 
                'avg_transaction_size', 'sent_volume', 
                'received_volume'
            ])
            writer.writeheader()
            writer.writerows(processed_whales)
            
        return filename

# Configuración y Ejecución del Análisis de Whales

Este código configura e inicia el análisis de whales:

- Configura análisis para últimos 30 días
- Define directorio de salida: "whale_reports"
- Ejecuta análisis completo
- Genera reporte con estadísticas de las whales

El reporte resultante contiene análisis de transacciones, balances y métricas de comportamiento para las top 100 wallets por balance.

In [25]:
config = TransactionConfig(
    output_dir="whale_reports",
    start_date=(datetime.now() - timedelta(days=30)).strftime("%Y-%m-%d"),
    end_date=datetime.now().strftime("%Y-%m-%d")
)

whale_analyzer = WhaleAnalyzer(config)
report_path = whale_analyzer.generate_whale_report()

2024-12-27 16:43:56,516 - __main__ - INFO - Obtenidas 6 transacciones adicionales
2024-12-27 16:44:00,542 - __main__ - INFO - Obtenidas 72 transacciones adicionales
2024-12-27 16:44:01,321 - __main__ - INFO - Obtenidas 33 transacciones adicionales
2024-12-27 16:44:02,121 - __main__ - INFO - Obtenidas 13 transacciones adicionales
2024-12-27 16:44:02,889 - __main__ - INFO - Obtenidas 5 transacciones adicionales
2024-12-27 16:44:03,715 - __main__ - INFO - Obtenidas 91 transacciones adicionales
2024-12-27 16:44:04,435 - __main__ - INFO - Obtenidas 6 transacciones adicionales
2024-12-27 16:44:05,127 - __main__ - INFO - Obtenidas 14 transacciones adicionales
2024-12-27 16:44:05,881 - __main__ - INFO - Obtenidas 5 transacciones adicionales
2024-12-27 16:44:07,692 - __main__ - INFO - Obtenidas 9 transacciones adicionales
2024-12-27 16:44:09,441 - __main__ - INFO - Obtenidas 2 transacciones adicionales
2024-12-27 16:44:10,596 - __main__ - INFO - Obtenidas 26 transacciones adicionales
2024-12-27